<h1 style="color:Black; text-align:center; font-size:3em;">Segmenting and Clustering Neighbourhoods in Toronto</h1>
<h3 style="color:Black; text-align:center; font-size:2em;">Scraping data, Creating dataframe, Analizing data, Clustering<h3>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 25px">

<font size = 2>

### 1. Build a dataframe to use in a clustering project with Foursquare location data

1a. <a href="#item1">Scrap Wikipedia</a>

1b. <a href="#item2">Build a table of the Toronto´s Neighbourhoods with its Postal code</a>

1c. <a href="#item3">Create a dataframe with the previous table</a>

1d. <a href="#item4">Clean the previous dataframe</a>

### 2. Assign the latitude and longitude coordinates for each Neighbourhood in the previous dataframe 

2a. <a href="#item4">Complete the dataframe: get the latitude and the longitude coordinates for each Neighbourhood</a>  

</font>
</div>

<h3 style="color:Black; text-align:left">This the new Notebook for this project</h3>

<div class="alert alert-block alert-info">
<b>Tip:</b> Notebook Ready.
</div>

***

### 1. Build a dataframe to use in a clustering project with Foursquare location data

<h4 style="color:Blue; text-align:left">1a. Scraping Wikipedia</h4>

This the url: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


Let´s build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [2]:
# Wikipedia url
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipage= requests.get(url).text

# Reading Wikipedia
wiki_data= BeautifulSoup(wikipage,'xml')
# print(wiki_data.prettify())

<h4 style="color:Blue; text-align:left">1b. Building the table</h4>

In [3]:
# Extracting data / building table

nh_table = wiki_data.find('table', class_ = 'wikitable sortable')
neightborhs = nh_table.find_all('tr')

neigh_table = []
for row in neightborhs:
    dat = row.text.split('\n')[1:-1]
    neigh_table.append(dat)
    
neigh_table[:5], print('This is the table of Toronto´s Postcodes')

This is the table of Toronto´s Postcodes


([['Postcode', 'Borough', 'Neighbourhood'],
  ['M1A', 'Not assigned', 'Not assigned'],
  ['M2A', 'Not assigned', 'Not assigned'],
  ['M3A', 'North York', 'Parkwoods'],
  ['M4A', 'North York', 'Victoria Village']],
 None)

<h4 style="color:Blue; text-align:left">1c. Creating the dataframe</h4>

In [4]:
# Building DataFrame

nh_df = pd.DataFrame(neigh_table[1:], columns=neigh_table[0])
print('This is the Dataframe of Toronto´s Postcodes'),
nh_df.head()

This is the Dataframe of Toronto´s Postcodes


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [388]:
# Shape of the initial dataframe

print('The shape of the initial dataframe is: {}'.format(nh_df.shape))

The shape of the initial dataframe is: (288, 3)


<div class="alert alert-block alert-info">
<b>Tip:</b> Dataframe Ready. Let´s clean it.
</div>

***

<h4 style="color:Blue; text-align:left">1d. Cleaning the dataframe</h4>

In [6]:
# Filtering out the rows that contain "Not assigned" in the 'Borough' Column

nh_df = nh_df[nh_df['Borough'] != 'Not assigned']

print('This is the shape of the dataframe wihtout "Not assigned" in the Borough Column : {}'.format(nh_df.shape))

This is the shape of the dataframe wihtout "Not assigned" in the Borough Column : (211, 3)


In [7]:
# Combining Neighbourhoods with the same Postcode into one row

def nh_list(grouped):
    return ', '.join(sorted(grouped['Neighbourhood'].tolist()))
                    
nh_df = nh_df.groupby(['Postcode', 'Borough'])
nhood_df = nh_df.apply(nh_list).reset_index(name='Neighbourhood')
nhood_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
print('This is the shape of the dataframe after combining rows: {}'.format(nhood_df.shape))

This is the shape of the dataframe after combining rows: (103, 3)


In [9]:
# Replacing "Not assigned" with "Borough" in the Neighbourhood column

nhood_df.loc[nhood_df['Neighbourhood']=="Not assigned",'Neighbourhood'] = nhood_df.loc[nhood_df['Neighbourhood']=="Not assigned",'Borough']
nhood_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


***
<h3 style="color:Blue; text-align:center; font-size:1em">1. This is the dataframe of Toronto´s Neighbourhoods+Postcodes</h3>

***

This is the Dataframe of Toronto´s Neighbourhood+Postcode ready to assign the latitude and the longitude coordinates

In [10]:
nhood_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
print('And this is the shape of this dataframe: {}'.format(nhood_df.shape))

And this is the shape of this dataframe: (103, 3)


### 2. Assign the latitude and longitude coordinates for each Neighbourhood in the previous dataframe 

<h4 style="color:Blue; text-align:left">2a. Getting the latitude and the longitude coordinates for each Neighbourhood</h4>

Downloading the dataset and reading it into a pandas dataframe:

In [12]:
nhCoord_df = pd.read_csv('https://cocl.us/Geospatial_data')
nhCoord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# Updating columns names, changing "Postal Code" to "Postcode", in order to merge the two dataframes

nhCoord_df.columns=['Postcode','Latitude','Longitude']
nhCoord_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let´s assign the logitude and latitude coordinates to each Neighbourhood

In [16]:
# Merging the two dataframes

TNh_df = pd.merge(nhood_df, nhCoord_df[['Postcode','Latitude', 'Longitude']], on='Postcode')

***
<h3 style="color:Blue; text-align:center; font-size:1em">2. The following is the dataframe of Toronto´s Neighbourhoods+Coordinates</h3>

***

<div class="alert alert-block alert-info">
<b>Tip:</b> The Dataframe is Ready to work with Foursquare data
</div>

***

In [17]:
TNh_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
print('This is the shape of this dataframe: {}'.format(TNh_df.shape))

This is the shape of this dataframe: (103, 5)
